In [ ]:
#!pip install openai
import openai
import pandas as pd
import time

In [ ]:
# Collect the product names from inventory csv files

df_start_inv= pd.read_csv("BegInvFINAL12312016.csv")
product_names_1= df_start_inv.Description.unique().tolist()
df_end_inv= pd.read_csv("EndInvFINAL12312016.csv")
product_names_2= df_end_inv.Description.unique().tolist()
product_names = product_names_1 + product_names_2
product_names= list(set(product_names))
print("Total Unique Products:", len(product_names))

In [ ]:
# Set up the GPT API
APIKEY= "" # Enter API key
openai.api_key = APIKEY

In [ ]:
import openai

APIKEY = "Enter API Key"
query= "What is the Alcohol By Type (ABV) of Jagermeister Liqueur?"

openai.api_key= APIKEY
response = openai.chat.completions.create(
    model="gpt-3.5-turbo", 
    messages=[{
        "role": "user", 
        "content": query}]
    )

# 1. ABV

In [ ]:
# Query through each product and collect ABV (Alcohol By Volume)

product_id=list()
products=list()
ABV=list()
error_products_id=list()
error_products=list()
error_products_error=list()

id=0

for product_name in product_names[id:]:
    query = "What is the ABV of "+product_name+ "? Also only give me the number but no text. For example, for the prompt 'What is the ABV of Jagermeister Liqueur?' response is '35'"
    output = openai.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[{"role": "user", 
                   "content": query
                   }])
    result= output.choices[0].message.content
    
    try:
        result_final= float(result)
        product_id.append(id)
        products.append(product_name)
        ABV.append(result_final)
        print(id,end=' ')
    except:
        error_products_id.append(id)
        error_products.append(product_name)
        error_products_error.append(result)
        print("\nError= "+product_name+" -> "+ result+ "\n")
    id+=1
        

In [ ]:
# Store successful queries 
result_df=pd.read_excel('product_desc.xlsx')

new_product_info=pd.DataFrame({
    "product_id":product_id,
    "product_name":products,
    "abv":ABV
})

result_df= result_df.append(new_product_info)
result_df.to_excel('product_desc.xlsx',index=False)

# Store unsuccessful queries 
error_df=pd.read_excel('product_desc_error.xlsx')

new_product_info=pd.DataFrame({
    "product_id":error_products_id ,
    "product_name":error_products,
    "abv_error":error_products_error
})

error_df= error_df.append(new_product_info)
error_df.to_excel('product_desc_error.xlsx',index=False)

# 2. Alcohol Type (whiskey, vodka, rum, tequila, gin, Liqueurs, Wine, Beer, Cider, Fortified wines, non-alcoholic, other)

In [ ]:
# Store successful queries 
def save_op(product_id, products, AT):
    result_df=pd.read_excel('product_desc.xlsx')
    new_product_info=pd.DataFrame({
        "product_id":product_id,
        "product_name":products,
        "alcohol_type":AT
    })
    result_df= result_df.append(new_product_info)
    result_df.to_excel('product_desc.xlsx',index=False)
    print("DF Save Success")

# Query through each product and collect ABV (Alcohol By Volume)
product_id=list()
products=list()
AT=list()

id=0

for product_name in product_names[id:]:
    query = 'What is the alcoholic type of'+product_name+'? Choose one from [whiskey, vodka, rum, tequila, gin, Liqueurs, Wine, Beer, Cider, Fortified wines, non-alcoholic, other]. Example response: "Liqueurs"'
    output = openai.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[{"role": "user", 
                   "content": query
                   }])
    result= output.choices[0].message.content

    # If the value returned is numeric as expected 
        #    if(result.isnumeric() or result.replace('.','',1).isnumeric()):
    result_final= result
    product_id.append(id)
    products.append(product_name)
    AT.append(result_final)
    print(id,end=' ')
    if(id%1000 == 0): 
        save_op(product_id, products, AT)
        product_id=list()
        products=list()
        AT=list()
    id+=1

save_op(product_id, products, AT)

# 3. Flavor Profile [Fruity, Spicy, Herbal, Floral, Sweet, Bitter, Sour, Savory, Other]

In [ ]:
avl_id= pd.read_excel("Flavor_profile.xlsx")
avl_id= avl_id.product_id.unique()
avl_id

In [ ]:
# Store successful queries 
def save_op(product_id, products, info):
    result_df=pd.read_excel('Flavor_profile.xlsx')
    new_product_info=pd.DataFrame({
        "product_id":product_id,
        "product_name":products,
        "flavor":info
    })
    result_df= result_df.append(new_product_info)
    result_df.to_excel('Flavor_profile.xlsx',index=False)
    print("DF Save Success")

# Query through each product and collect ABV (Alcohol By Volume)
product_id=list()
products=list()
info=list()

id=8097

#for product_name in product_names[id:]:
for i in range (id,9842):
    product_name= product_names[i]
    if(i in avl_id): 
        continue
    query = 'What is the flavour profile of '+product_name+' ? Choose one from [Fruity, Spicy, Herbal, Floral, Sweet, Bitter, Sour, Savory, Other]. Example response: "Herbal"'

    output = openai.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=[{"role": "user", 
                   "content": query
                   }])
                   
    result= output.choices[0].message.content

    result_final= result
    product_id.append(i)
    products.append(product_name)
    info.append(result_final)
    print(i,end=' ')

    if(i%15 == 0):
        print("Pausing for 30 seconds...", end= " ")
        time.sleep(30)
        print("Resuming...")

    if(i%1000 == 0): 
        save_op(product_id, products, info)
        product_id=list()
        products=list()
        info=list()

save_op(product_id, products, info)